# Basics of simple fully connected neural network 

In [1]:
# import required libraries
import pandas as pd
import numpy as np
import keras
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
# LabelEncoder is important for neural network analysis for converting output labels array
# into one hot encoded matrix of K columns where K is the unqiue number of categories
%matplotlib inline

C:\Users\Alvin\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# read in data
# dataset_redwine = pd.read_csv('data/redwine_data.csv',sep=';')
dataset_redwine = pd.read_csv('redwine_fully_NN.csv',sep=';')
dataset_redwine.head()


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [3]:
# data preprocessing
input_features = dataset_redwine.iloc[:,0:11].values
targets = dataset_redwine.iloc[:,11].values
encoder = LabelEncoder()
y_transformed = encoder.fit_transform(targets) # convert labels into a series of integers
exact_labels = pd.get_dummies(y_transformed).values # convert series of integers into one hot 
# encoded matrix

In [4]:
# split data into training and testing dataset
train_data,test_data,train_target,test_target = train_test_split(input_features,
                                                                 exact_labels,
                                                                 test_size = 0.2,
                                                                 random_state = 0)

In [5]:
# scaling the dataset for ANN modelling
features_scaler = StandardScaler()
scaled_train_data = features_scaler.fit_transform(train_data)
scaled_test_data = features_scaler.transform(test_data)

In [6]:
# setting up fully connected ANN model
model_ANN = Sequential() # initialize model
input_layer = Dense(units = 11, kernel_initializer = 'uniform',
                   activation = 'relu', input_dim = 11) # input layer, kernel_initializer set 
# the weights; uniform -> uniform distribution, activation function of relu -> rectifier function
model_ANN.add(input_layer)

hidden_layer1 = Dense(units = 9, kernel_initializer = 'uniform',
                   activation = 'relu') # first hidden layer
model_ANN.add(hidden_layer1)

hidden_layer2 = Dense(units = 7, kernel_initializer = 'uniform',
                   activation = 'relu') # second hidden layer
model_ANN.add(hidden_layer2)

output_layer = Dense(units = 6, kernel_initializer = 'uniform',
                    activation = 'softmax') # softmax is used for multiple categories while 
# sigmoid is for binary output
model_ANN.add(output_layer)

In [7]:
# compile ANN model
model_ANN.compile(optimizer = 'adam', # see more about optimization function 
                                      # https://keras.io/optimizers/
                 loss = 'categorical_crossentropy', # see more about cost functions
                                                    # https://keras.io/losses/
                 metrics = ['accuracy'])
model_ANN.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 11)                132       
_________________________________________________________________
dense_2 (Dense)              (None, 9)                 108       
_________________________________________________________________
dense_3 (Dense)              (None, 7)                 70        
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 48        
Total params: 358
Trainable params: 358
Non-trainable params: 0
_________________________________________________________________


In [8]:
# fit ANN model
model_ANN.fit(scaled_train_data,
              train_target,
              batch_size = 1,
              epochs = 10,
              verbose = 1,
              validation_data = (scaled_test_data,test_target))

Train on 1279 samples, validate on 320 samples
Epoch 1/10
1279/1279 [==============================] - 3s 2ms/step - loss: 1.2449 - acc: 0.4246 - val_loss: 1.1002 - val_acc: 0.4875
Epoch 2/10
1279/1279 [==============================] - 2s 2ms/step - loss: 1.1036 - acc: 0.5098 - val_loss: 1.0716 - val_acc: 0.5750
Epoch 3/10
1279/1279 [==============================] - 2s 2ms/step - loss: 1.0768 - acc: 0.5465 - val_loss: 1.0514 - val_acc: 0.6094
Epoch 4/10
1279/1279 [==============================] - 3s 2ms/step - loss: 1.0573 - acc: 0.5606 - val_loss: 1.0384 - val_acc: 0.6062
Epoch 5/10
1279/1279 [==============================] - 2s 2ms/step - loss: 1.0400 - acc: 0.5715 - val_loss: 1.0246 - val_acc: 0.6281
Epoch 6/10
1279/1279 [==============================] - 2s 2ms/step - loss: 1.0256 - acc: 0.5629 - val_loss: 1.0132 - val_acc: 0.6156
Epoch 7/10
1279/1279 [==============================] - 3s 2ms/step - loss: 1.0130 - acc: 0.5622 - val_loss: 1.0167 - val_acc: 0.6250
Epoch 8/10
1279

In [9]:
# test model's accuracy on test dataset
pred_data = model_ANN.predict(scaled_test_data)
pred_data = np.argmax(pred_data,axis = 1)
test_target = np.argmax(test_target, axis =1)
score = accuracy_score(test_target,pred_data) * 100
score = round(score,1)
print('The average accuracy of your fully-connected neural network model is ' + str(score) + '%.')

The average accuracy of your fully-connected neural network model is 61.3%.
